In [89]:
import numpy as np 
import pandas as pd 

In [90]:
fake_df=pd.read_csv("Fake.csv")
true_df=pd.read_csv("True.csv")

In [91]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [92]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [93]:
fake_df.subject.unique(),true_df.subject.unique()


(array(['News', 'politics', 'Government News', 'left-news', 'US_News',
        'Middle-east'], dtype=object),
 array(['politicsNews', 'worldnews'], dtype=object))

## Cat't use subject to train our model
So we can't use subject to train our model. If we use it <br>
our model is going to get overfitted and will give poor accuracy, <br>
because we have complete diffrent subject in fake_df and true_df

In [94]:
fake_df["IsFake"]=1
true_df['IsFake']=0

In [95]:
train_df=pd.concat((true_df,fake_df),axis=0)

In [96]:
train_df.isnull().sum()

title      0
text       0
subject    0
date       0
IsFake     0
dtype: int64

### we have no null values

In [97]:
train_df[train_df.duplicated()]

,title,text,subject,date,IsFake
445,Senate tax bill stalls on deficit-focused 'tri...,WASHINGTON (Reuters) - The U.S. Senate on Thur...,politicsNews,"November 30, 2017",0
778,Trump warns 'rogue regime' North Korea of grav...,BEIJING (Reuters) - U.S. President Donald Trum...,politicsNews,"November 8, 2017",0
892,"Republicans unveil tax cut bill, but the hard ...",WASHINGTON (Reuters) - U.S. House of Represent...,politicsNews,"November 2, 2017",0
896,Trump taps Fed centrist Powell to lead U.S. ce...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"November 2, 2017",0
974,"Two ex-Trump aides charged in Russia probe, th...",WASHINGTON (Reuters) - Federal investigators p...,politicsNews,"October 30, 2017",0
...,...,...,...,...,...
21353,Thailand's ousted PM Yingluck has fled abroad:...,BANGKOK (Reuters) - Ousted Thai prime minister...,worldnews,"August 25, 2017",0
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",0
9942,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017",1
11446,FORMER DEMOCRAT WARNS Young Americans: “Rioter...,"Who is silencing political speech, physically...",politics,"Mar 10, 2017",1


### we have 209 duplicate values

In [98]:
# Let's drop duplicates
train_df.drop_duplicates(inplace=True)

In [99]:
train_df.IsFake.value_counts()

IsFake
1    23478
0    21211
Name: count, dtype: int64

### We have a almoast balanced dataset so maybe we don't have to do down sampling/up sampling

In [100]:
train_df["title_text"]=train_df.title+train_df.text

In [101]:
dataset=train_df.drop(["text","title","subject"],axis=1)

In [102]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to C:\Users\IT
[nltk_data]     TECH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [103]:
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')
def clean_text(textset):
    corpus=[]
    for sen in textset:
        text=re.sub('[^A-Za-z]',' ',sen)
        text=text.lower()
        text=text.split()
        ps=PorterStemmer()
        text=[ps.stem(word) for word in text if word not in set(all_stopwords)]
        text=' '.join(text)
        corpus.append(text)
    return np.array(corpus)

In [104]:
dataset.title_text=clean_text(dataset.title_text)

In [105]:
dataset.to_csv('clean_text_news.csv',index=False)